The need to interactively tune the fiducial marker detection and check its output means that drift correction should not be performed in batch. However, the drift correction process can be made simpler by using a notebook that assists in finding the files and performing the drift correction. This notebook serves as the template for this step.

Note: Typically, this step would be performed before the actual batch processing in tutorial 2. Since tutorial 2 was easier, though, I thought it would be best to explain it before this one.

### Import the software libraries

In [1]:
%pylab
import DataSTORM.processors as proc
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Outline of steps
1. Search the parent directory for all localization files and a build a list of such files
2. Define the drift correction processor
3. Open a file from the list
4. Perform the drift correction on the file
  * If the correction was not good, go back to step 2
4. Save the results
5. Repeat from step 1

# Search the parent directory and make a list of localization files
We will start by searching a parent directory and its subdirectories for all the localization files, i.e. those files that end in `locResults.dat`.

In [2]:
parentDirectory   = Path('../test-data/Centrioles/')
localizationFiles = parentDirectory.glob('**/*locResults.dat')
locResultFiles    = sorted(localizationFiles)

# How many files are there?
print(len(locResultFiles))

2


`locResultFiles` is an array of files ending in `locResults.dat` within the parent directory `../test-data/Centrioles/` and its subdirectories.

In [3]:
locResultFiles[0] # In Python, array indexes start at 0

PosixPath('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults.dat')

In [4]:
locResultFiles[1]

PosixPath('../test-data/Centrioles/FOV_7_noPB_1500mW_10ms_1/FOV_7_noPB_1500mW_10ms_1_MMStack_locResults.dat')

# Define the drift correction processor
This part is the same as in Tutorial 1. At the end of processing for each file, return to this code block and start running the blocks again.

In [5]:
# Create the FiducialDriftCorrect processor and set its properties.
dc = proc.FiducialDriftCorrect(minFracFiducialLength = 0.5,
                               neighborRadius        = 500,
                               smoothingWindowSize   = 2000,
                               smoothingFilterSize   = 500,
                               interactiveSearch     = True,
                               noLinking             = True,
                               noClustering          = True)

clean = proc.CleanUp()

# Open a file from the list
Now, we're going to open the first file in this list, locResultFiles[0]. When we are all done with this file, we will come back to here and increase the index by one (to locResultFiles[1], and so on...).

In [6]:
# Set the input and output files
currentFile = locResultFiles[0] # Increment this when done dedrifting a dataset
outputFile  = currentFile.parent / Path(currentFile.stem + '_DC' + currentFile.suffix)

# Open a file and clean it up
with open(str(currentFile), 'r') as file:
    df = pd.read_csv(file)

# Clean up the data
df = clean(df)

In [7]:
df.describe()

,x [nm],y [nm],z [nm],frame,uncertainty [nm],intensity [photon],offset [photon],loglikelihood,sigma [nm]
count,1120018.000000,1120018.000000,1120018,1120018.000000,1120018.000000,1120018.000000,1120018.000000,1120018.000000,1120018.000000
mean,24970.090713,31160.454307,0,29187.099778,766.523967,4139.249990,186.332021,228.356764,138.228309
std,16545.141548,18919.796124,0,24342.325833,10230.296829,2868.737803,38.787012,436.768610,23.849260
min,85.238000,0.106440,0,100.000000,0.632450,1.000000,68.953000,-37.829000,54.000000
25%,9251.300000,11961.000000,0,6803.000000,3.752500,2290.300000,167.600000,86.804000,127.090000
50%,24876.000000,27527.000000,0,23345.000000,5.152900,3261.100000,179.600000,118.520000,134.690000
75%,36370.000000,51665.000000,0,49176.000000,6.637600,5090.200000,193.180000,198.420000,143.700000
max,65011.000000,65004.000000,0,79999.000000,172430.000000,67290.000000,1315.400000,33596.000000,378.000000


# Perform the drift correction
Now that the file is open we can perform the drift correction in the same manner as on a typical file.

In [8]:
corrdf = dc(df)

1 fiducial(s) detected.
Performing spline fits...


/home/douglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/matplotlib/backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [9]:
dc.plotFiducials()

At this point, you have to check whether the fiducial trajectories are good enough to give a spline fit with the `plotFicuials()` function above. If they are not, you can cycle back to the definition of the corrector, change some of the parameters, and then rerun the steps.

The general workflow for tuning the drift correction module is:

1. Try to identify long-lived fiducials without using linking or clustering.
2. If the fiducial tracks are noisy, enable linking and tune the linking parameters.
3. If the fiducial tracks are still too noisy, enable clustering and tune the clustering parameters. (But only if there are fewer than 50,000 frames! Otherwise, clustering will eat up all the memory in the computer.)
4. If you can't get any good fiducials, reload the DataFrame and skip the drift correction steps. Alternatively, if no fiducials were officially found in the linking/clustering, run the code below to save the file with a _DCX suffix.

In [10]:
if len(dc.fiducialTrajectories) == 0:
    with open(str(currentFile), 'r') as file:
        df = pd.read_csv(file)

    df = clean(df)

    outputFile = currentFile.parent / Path(currentFile.stem + '_DCX' + currentFile.suffix)

In [11]:
# Save the data
with open(str(outputFile), 'w') as file:
    df.to_csv(file, index=False)

Now you can cycle back up to the beginning and increase the index locResultFiles array by one and repeat the process in this notebook.